In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from transformers import pipeline
import spacy

from epub import EPub


In [ ]:
this_file = Path(__name__).absolute()
snippet_folder = this_file.parent.parent.parent.parent
dataset_folder = snippet_folder / "datasets" / "ebook"
epub_path = {
    "fr": dataset_folder / "Gaston_Leroux_-_Le_Mystere_de_la_chambre_jaune.epub",
    "en": dataset_folder / "mystery_yellow_room.epub",
}
print(epub_path)


In [ ]:
nlp = {
    "en": spacy.load("en_core_web_md"),
    "fr": spacy.load("fr_core_news_md"),
}


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
pipe_fren = pipeline("translation", model=model_checkpoint)


In [ ]:
lts = list(nlp.keys())
lts


In [ ]:
epub = {lt: EPub(epub_path[lt], nlp[lt]) for lt in lts}


In [ ]:
sent_fr = epub["fr"].chapters[1].paragraphs[0].sentences[0]
sent_fr.text


In [ ]:
sent_fr_to_en = pipe_fren(sent_fr.text)
sent_fr_to_en


In [ ]:
doc_fr_to_en = nlp["en"](sent_fr_to_en[0]["translation_text"])
print(type(doc_fr_to_en))
doc_fr_to_en


In [ ]:
sent_en = epub["en"].chapters[0].paragraphs[2].sentences[0]
print(type(sent_en))
sent_en


In [ ]:
doc_fr_to_en.similarity(sent_en)
